# Project 2 - Ames Housing Top Features 
____
____

## Contents:
___
* [Summary](#Summary)
* [Problem Statement](#Problem-Statement)
* [Resources](#Resources)
* [Import Libraries](#Import-Libraries)
* [Import Data](#Import-Data)
* [Pre-Exploratory Data Analysis](#Pre-Exploratory-Data-Analysis)
* [Data Cleaning](#Data-Cleaning)
___

## Summary
___
The Ames Housing Data Set contains information from the Ames Assessor’s Office used in computing the value of individual residential properties sold in Ames, Iowa from 2006 to 2010, as well as the actual eventual sale prices for the properties.

The data set contains information for more than 2000 properties. The initial data dictionary outlines, which will not be illustrated, have more than 75 descriptive variables. Some are **nominal** (categorical), meaning they are non-numerical and lack clear-cut order (IE: Neighborhood, Type of roofing). Some are **ordinal**, meaning they are categorical but have a clear order (IE: External Quality (Excellent, Good, Average, Poor)). Some are **discrete**, meaning they are numerical but at set intervals (IE: Year Built, Garage Cars). The rest are **continuous**, meaning they are numerical and can theoretically take any value in a range (IE: 1st Floor Square Feet, 2nd Floor Square Feet, Basement Square Feet).

In this project, I have attempted to craft as accurate of a model as possible for predicting housing sale prices, using regression techniques, enhanced by feature engineering, feature selection, and regularization.The Ames data set was divided into a training set and a test set. I aim to accurately predict the withheld sale prices for the test set, based on the features and prices in the training set. 

[^ Return to Contents](#Contents:)

## Problem Statement
___
The main purpose of this project is to predict the home prices in Ames, Iowa by using regression models. After doing this data analysis, theoretical relationship can be found with what we have in mind. Throughout the progressive iterative modeling and feature selection processes in both manually and automatic, we can gain a deeper insight into variables which were directly correltes to property sales and understand better the mechanism behind various models.

[^ Return to Contents](#Contents:)

## Resources
___
To get a better understanding on what resides in Ames, Iowa, videos and online sites provided additional research materials. Sources from reputable YouTube content creators to websites in real estate, best places to live, and others were used to add values on top of the provide data.

**Data:**
* [Train CSV](train.csv)
* [Test CSV](test.csv)
* [Complete Dictionary](../datasets/data_description.txt)

**3rd Party**
- [The 2020 Top 100 Best Places to Live in America](https://livability.com/best-places/top-100-best-places-to-live/2020/ames-ia/)
- [Quickloan](https://www.quickenloans.com/learn/things-look-shopping-next-home)
- [Top House-Hunting Mistakes](https://www.investopedia.com/articles/mortgage-real-estate/09/buy-house-emotion-free.asp)
- [Iowa Home Buyer's Handbook](https://www.greatiowahomes.com/buyers/handbook.cfm)

[^ Return to Contents](#Contents:)

# Import Libraries
___
Importing the necessary libraries that allow the use of the functions
___

In [1800]:
#import libraries
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns

In [1801]:
# Enables Pandas to display all the columns
pd.set_option('display.max_columns', None)

# Enables Pandas to display all the rows
pd.set_option('display.max_rows', None)

In [1802]:
# import sklearn
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn import metrics
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')

In [1803]:
# able to see graph
%matplotlib inline 

[^ Return to Contents](#Contents:)

# Import CSV files
___
Importing the necessary files to help find the answers to the above problems.
___

In [1804]:
#create new instances and import dataset
test_df = pd.read_csv('../Data/test.csv')

In [1805]:
#create a backup
backup_test_df = test_df

[^ Return to Contents](#Contents:)

# Sneak Peek: Understand What is Inside
___
Prior to fixing the file, lets understand what we are dealing with so can get a good idea how and where to start.
___

In [1806]:
#see sample size
test_df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2Story,6,8,1910,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,TA,Fa,Stone,Fa,TA,No,Unf,0,Unf,0,1020,1020,GasA,Gd,N,FuseP,908,1020,0,1928,0,0,2,0,4,2,Fa,9,Typ,0,NaN,Detchd,1910.0,Unf,1,440,Po,Po,Y,0,60,112,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,Duplex,1Story,5,4,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,Gd,TA,No,Unf,0,Unf,0,1967,1967,GasA,TA,Y,SBrkr,1967,0,0,1967,0,0,2,0,6,2,TA,10,Typ,0,NaN,Attchd,1977.0,Fin,2,580,TA,TA,Y,170,0,0,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,Gd,Av,GLQ,554,Unf,0,100,654,GasA,Ex,Y,SBrkr,664,832,0,1496,1,0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,2006.0,RFn,2,426,TA,TA,Y,100,24,0,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1Story,5,6,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,Gd,TA,CBlock,TA,TA,No,Unf,0,Unf,0,968,968,GasA,TA,Y,SBrkr,968,0,0,968,0,0,1,0,2,1,TA,5,Typ,0,NaN,Detchd,1935.0,Unf,2,480,Fa,TA,N,0,0,184,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,TA,TA,CBlock,Gd,TA,No,BLQ,609,Unf,0,785,1394,GasA,Gd,Y,SBrkr,1394,0,0,1394,1,0,1,1,3,1,TA,6,Typ,2,Gd,Attchd,1963.0,RFn,2,514,TA,TA,Y,0,76,0,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [1807]:
#see train info
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               878 non-null    int64  
 1   PID              878 non-null    int64  
 2   MS SubClass      878 non-null    int64  
 3   MS Zoning        878 non-null    object 
 4   Lot Frontage     718 non-null    float64
 5   Lot Area         878 non-null    int64  
 6   Street           878 non-null    object 
 7   Alley            58 non-null     object 
 8   Lot Shape        878 non-null    object 
 9   Land Contour     878 non-null    object 
 10  Utilities        878 non-null    object 
 11  Lot Config       878 non-null    object 
 12  Land Slope       878 non-null    object 
 13  Neighborhood     878 non-null    object 
 14  Condition 1      878 non-null    object 
 15  Condition 2      878 non-null    object 
 16  Bldg Type        878 non-null    object 
 17  House Style     

[^ Return to Contents](#Contents:)

# Pre-Exploratory Data Analysis
___
Now having a general idea what are within both Train and Test CSV files, now to explore it deeper before prior cleaning the datas. Since there are both numerical and string formats within the Train data, it would be easier to see which columns belong to which by grouping them together.
___

In [1808]:
#group columns that are int/float type
num_cols = list(test_df.select_dtypes(exclude='object').columns)

#display int/float columns
print(num_cols)

['Id', 'PID', 'MS SubClass', 'Lot Frontage', 'Lot Area', 'Overall Qual', 'Overall Cond', 'Year Built', 'Year Remod/Add', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Fireplaces', 'Garage Yr Blt', 'Garage Cars', 'Garage Area', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val', 'Mo Sold', 'Yr Sold']


In [1809]:
#number of numerial columns
print(f"There are {len(num_cols)} numerial columns")

There are 38 numerial columns


In [1810]:
#group columns that are string type
obj_cols = list(test_df.select_dtypes(include='object').columns)

#display obj columns
print(obj_cols)

['MS Zoning', 'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating', 'Heating QC', 'Central Air', 'Electrical', 'Kitchen Qual', 'Functional', 'Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond', 'Paved Drive', 'Pool QC', 'Fence', 'Misc Feature', 'Sale Type']


In [1811]:
#number of object columns
print(f"There are {len(obj_cols)} string columns")

There are 42 string columns


[^ Return to Contents](#Contents:)

# Data Cleaning
___
Now having an idea what to look into to get a better gauge of what to look out for, lets see if there are any improvement after cleaning the data
___

In [1812]:
#change column format to better code
test_df.columns = test_df.columns.str.replace(' ','')

In [1813]:
#see sample of output
test_df.head()

,Id,PID,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemod/Add,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2Story,6,8,1910,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,TA,Fa,Stone,Fa,TA,No,Unf,0,Unf,0,1020,1020,GasA,Gd,N,FuseP,908,1020,0,1928,0,0,2,0,4,2,Fa,9,Typ,0,NaN,Detchd,1910.0,Unf,1,440,Po,Po,Y,0,60,112,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,Duplex,1Story,5,4,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,Gd,TA,No,Unf,0,Unf,0,1967,1967,GasA,TA,Y,SBrkr,1967,0,0,1967,0,0,2,0,6,2,TA,10,Typ,0,NaN,Attchd,1977.0,Fin,2,580,TA,TA,Y,170,0,0,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,Gd,Av,GLQ,554,Unf,0,100,654,GasA,Ex,Y,SBrkr,664,832,0,1496,1,0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,2006.0,RFn,2,426,TA,TA,Y,100,24,0,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1Story,5,6,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,Gd,TA,CBlock,TA,TA,No,Unf,0,Unf,0,968,968,GasA,TA,Y,SBrkr,968,0,0,968,0,0,1,0,2,1,TA,5,Typ,0,NaN,Detchd,1935.0,Unf,2,480,Fa,TA,N,0,0,184,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,TA,TA,CBlock,Gd,TA,No,BLQ,609,Unf,0,785,1394,GasA,Gd,Y,SBrkr,1394,0,0,1394,1,0,1,1,3,1,TA,6,Typ,2,Gd,Attchd,1963.0,RFn,2,514,TA,TA,Y,0,76,0,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [1814]:
#find all the notable nulls
test_df.isnull().sum()

Id                 0
PID                0
MSSubClass         0
MSZoning           0
LotFrontage      160
LotArea            0
Street             0
Alley            820
LotShape           0
LandContour        0
Utilities          0
LotConfig          0
LandSlope          0
Neighborhood       0
Condition1         0
Condition2         0
BldgType           0
HouseStyle         0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemod/Add      0
RoofStyle          0
RoofMatl           0
Exterior1st        0
Exterior2nd        0
MasVnrType         1
MasVnrArea         1
ExterQual          0
ExterCond          0
Foundation         0
BsmtQual          25
BsmtCond          25
BsmtExposure      25
BsmtFinType1      25
BsmtFinSF1         0
BsmtFinType2      25
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
Heating            0
HeatingQC          0
CentralAir         0
Electrical         1
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea    

## Dropping
___
From skimming the list of nulls, columns 'Alley', 'Pool QC', 'Fence', 'Fireplace Qu', and 'Misc Feature' will go through further examination to determine if it can be dropped.

In [1815]:
#create a new instance of the test_df
new_test_df = test_df

In [1816]:
#see the types of uniqueness of the column
new_test_df['Alley'].value_counts(dropna=False)

NaN     820
Grvl     35
Pave     23
Name: Alley, dtype: int64

In [1817]:
#see the types of uniqueness of the column
new_test_df['PoolQC'].value_counts(dropna=False)

NaN    874
Ex       3
TA       1
Name: PoolQC, dtype: int64

In [1818]:
#see the types of uniqueness of the column
new_test_df['Fence'].value_counts(dropna=False)

NaN      706
MnPrv    103
GdPrv     35
GdWo      32
MnWw       2
Name: Fence, dtype: int64

In [1819]:
#see the types of uniqueness of the column
new_test_df['FireplaceQu'].value_counts(dropna=False)

NaN    422
Gd     220
TA     193
Fa      16
Po      15
Ex      12
Name: FireplaceQu, dtype: int64

In [1820]:
#see the types of uniqueness of the column
new_test_df['MiscFeature'].value_counts(dropna=False)

NaN     837
Shed     39
Othr      1
Gar2      1
Name: MiscFeature, dtype: int64

___
After comparing the given unique values from each column.  FirePlaceQC can be salvage by stating that the houses with a null value have no fireplace built within.  This can be a new unique value within FirePlaceQC.  As for others, there's too much information missing that may skew the result if tried to impliment a substitue value in.  Those will be dropped.
___

In [1821]:
#create new dictionary for FireplaceQu
new_test_df['FireplaceQu'] = new_test_df['FireplaceQu'].fillna('NoFirePlace')

In [1822]:
#drop all other null columns
new_test_df.drop(['Alley','PoolQC', 'Fence', 'MiscFeature'], axis=1, inplace=True)
new_test_df.head()

,Id,PID,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemod/Add,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType
0,2658,902301120,190,RM,69.0,9142,Pave,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2Story,6,8,1910,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,TA,Fa,Stone,Fa,TA,No,Unf,0,Unf,0,1020,1020,GasA,Gd,N,FuseP,908,1020,0,1928,0,0,2,0,4,2,Fa,9,Typ,0,NoFirePlace,Detchd,1910.0,Unf,1,440,Po,Po,Y,0,60,112,0,0,0,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,IR1,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,Duplex,1Story,5,4,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,Gd,TA,No,Unf,0,Unf,0,1967,1967,GasA,TA,Y,SBrkr,1967,0,0,1967,0,0,2,0,6,2,TA,10,Typ,0,NoFirePlace,Attchd,1977.0,Fin,2,580,TA,TA,Y,170,0,0,0,0,0,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,Gd,Av,GLQ,554,Unf,0,100,654,GasA,Ex,Y,SBrkr,664,832,0,1496,1,0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,2006.0,RFn,2,426,TA,TA,Y,100,24,0,0,0,0,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1Story,5,6,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,Gd,TA,CBlock,TA,TA,No,Unf,0,Unf,0,968,968,GasA,TA,Y,SBrkr,968,0,0,968,0,0,1,0,2,1,TA,5,Typ,0,NoFirePlace,Detchd,1935.0,Unf,2,480,Fa,TA,N,0,0,184,0,0,0,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,TA,TA,CBlock,Gd,TA,No,BLQ,609,Unf,0,785,1394,GasA,Gd,Y,SBrkr,1394,0,0,1394,1,0,1,1,3,1,TA,6,Typ,2,Gd,Attchd,1963.0,RFn,2,514,TA,TA,Y,0,76,0,0,185,0,0,7,2009,WD


In [1823]:
#check out info
new_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             878 non-null    int64  
 1   PID            878 non-null    int64  
 2   MSSubClass     878 non-null    int64  
 3   MSZoning       878 non-null    object 
 4   LotFrontage    718 non-null    float64
 5   LotArea        878 non-null    int64  
 6   Street         878 non-null    object 
 7   LotShape       878 non-null    object 
 8   LandContour    878 non-null    object 
 9   Utilities      878 non-null    object 
 10  LotConfig      878 non-null    object 
 11  LandSlope      878 non-null    object 
 12  Neighborhood   878 non-null    object 
 13  Condition1     878 non-null    object 
 14  Condition2     878 non-null    object 
 15  BldgType       878 non-null    object 
 16  HouseStyle     878 non-null    object 
 17  OverallQual    878 non-null    int64  
 18  OverallCon

In [1824]:
#find all the notable nulls
test_df.isnull().sum()

Id                 0
PID                0
MSSubClass         0
MSZoning           0
LotFrontage      160
LotArea            0
Street             0
LotShape           0
LandContour        0
Utilities          0
LotConfig          0
LandSlope          0
Neighborhood       0
Condition1         0
Condition2         0
BldgType           0
HouseStyle         0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemod/Add      0
RoofStyle          0
RoofMatl           0
Exterior1st        0
Exterior2nd        0
MasVnrType         1
MasVnrArea         1
ExterQual          0
ExterCond          0
Foundation         0
BsmtQual          25
BsmtCond          25
BsmtExposure      25
BsmtFinType1      25
BsmtFinSF1         0
BsmtFinType2      25
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
Heating            0
HeatingQC          0
CentralAir         0
Electrical         1
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath 

## Null Values in LotFrontage
___


In [1825]:
#see the NaNs in lot frontage
new_test_df['LotFrontage'].value_counts(dropna=False)

NaN      160
60.0      97
80.0      43
75.0      37
70.0      37
50.0      27
85.0      24
65.0      22
21.0      18
24.0      16
68.0      16
90.0      15
78.0      13
64.0      12
51.0      11
55.0      10
79.0       9
63.0       9
76.0       9
59.0       9
72.0       9
52.0       8
73.0       8
86.0       8
61.0       8
74.0       8
66.0       7
44.0       7
40.0       7
82.0       7
53.0       6
110.0      6
69.0       6
71.0       6
88.0       6
57.0       6
120.0      6
35.0       6
100.0      5
34.0       5
48.0       5
98.0       5
54.0       4
94.0       4
56.0       4
84.0       4
93.0       4
89.0       4
36.0       4
58.0       4
42.0       4
81.0       4
95.0       4
77.0       4
67.0       4
43.0       3
96.0       3
105.0      3
99.0       3
102.0      3
83.0       3
124.0      3
62.0       3
41.0       3
87.0       3
118.0      3
149.0      2
107.0      2
47.0       2
39.0       2
49.0       2
45.0       2
33.0       2
32.0       2
115.0      2
104.0      2
92.0       2

**Observation:**
NaNs seem to be too highly represented for us to assume that we can simply replace all with 0.
City of Ames Municipal code does have regulations for minimum lot frontage.

Because the Lot Configuration has a close relationship with Lot Frontage. We can group them together to find a result to fill the null value.

In [1826]:
#check the mean lot frontage for each category of lot configuration
new_test_df.groupby('LotConfig', as_index=False)[['LotFrontage']].mean()

,LotConfig,LotFrontage
0,Corner,81.452381
1,CulDSac,59.727273
2,FR2,62.625000
3,FR3,48.500000
4,Inside,67.496377


In [1827]:
#check the median lot frontage for each category of lot configuration
new_test_df.groupby('LotConfig', as_index=False)[['LotFrontage']].median()

,LotConfig,LotFrontage
0,Corner,79.5
1,CulDSac,48.5
2,FR2,65.0
3,FR3,48.5
4,Inside,67.0


The values are quite different for each grouping. The median will be selected, since it is less sensitive to outliers.

In [1828]:
#fill in the NaN with the mean value of Lot Frontage
#new_test_df['LotFrontage'] = new_test_df['LotFrontage'].fillna(new_test_df['LotFrontage'].median())

new_test_df.loc[(new_test_df['LotFrontage'].isna()) & (new_test_df['LotConfig'] == 'Corner'), 'LotFrontage'] = 80
new_test_df.loc[(new_test_df['LotFrontage'].isna()) & (new_test_df['LotConfig'] == 'CulDSac'), 'LotFrontage'] = 50
new_test_df.loc[(new_test_df['LotFrontage'].isna()) & (new_test_df['LotConfig'] == 'FR2'), 'LotFrontage'] = 60
new_test_df.loc[(new_test_df['LotFrontage'].isna()) & (new_test_df['LotConfig'] == 'FR3'), 'LotFrontage'] = 80.0
new_test_df.loc[(new_test_df['LotFrontage'].isna()) & (new_test_df['LotConfig'] == 'Inside'), 'LotFrontage'] = 66.0

In [1829]:
#confirm there is no more null value for LotFrontage
new_test_df['LotFrontage'].isnull().sum()

0

### Null Values in MasVnrArea and MasVnrType

In [1830]:
#check the number of unique value in MasVnrType, including null value
new_test_df['MasVnrArea'].value_counts(dropna=False)

0.0       532
216.0       7
196.0       5
80.0        5
420.0       5
340.0       4
120.0       4
144.0       4
50.0        3
285.0       3
240.0       3
194.0       3
176.0       3
90.0        3
180.0       3
182.0       3
149.0       3
302.0       3
128.0       3
198.0       3
200.0       3
306.0       3
456.0       3
270.0       3
88.0        3
178.0       2
76.0        2
162.0       2
156.0       2
147.0       2
280.0       2
174.0       2
300.0       2
621.0       2
256.0       2
305.0       2
232.0       2
226.0       2
450.0       2
360.0       2
320.0       2
423.0       2
82.0        2
72.0        2
99.0        2
166.0       2
14.0        2
100.0       2
106.0       2
209.0       2
16.0        2
268.0       2
169.0       2
45.0        2
206.0       2
352.0       2
286.0       2
246.0       2
108.0       2
260.0       2
164.0       2
161.0       2
350.0       2
266.0       2
104.0       2
130.0       2
53.0        2
70.0        2
123.0       2
150.0       2
265.0       2
20.0  

In [1831]:
#check the number of unique value in MasVnrType, including null value
new_test_df['MasVnrType'].value_counts(dropna=False)

None       534
BrkFace    250
Stone       80
BrkCmn      12
NaN          1
CBlock       1
Name: MasVnrType, dtype: int64

In [1832]:
#to confirm there is a relationship between the two
new_test_df[new_test_df['MasVnrArea'].isna()].equals(new_test_df[new_test_df['MasVnrType'].isna()])

True

In [1833]:
#convert the null value to None value because it has the biggest percentage 
new_test_df['MasVnrType'] = new_test_df['MasVnrType'].replace(np.nan, 'None')

In [1834]:
#convert the null value to zero value because it has the biggest percentage
new_test_df['MasVnrArea'] = new_test_df['MasVnrArea'].replace(np.nan, 0.0)

In [1835]:
#confirm there is no more null value for LotFrontage
new_test_df['MasVnrArea'].isnull().sum()

0

In [1836]:
#confirm there is no more null value for LotFrontage
new_test_df['MasVnrType'].isnull().sum()

0

In [1837]:
#find all the notable nulls
test_df.isnull().sum()

Id                0
PID               0
MSSubClass        0
MSZoning          0
LotFrontage       0
LotArea           0
Street            0
LotShape          0
LandContour       0
Utilities         0
LotConfig         0
LandSlope         0
Neighborhood      0
Condition1        0
Condition2        0
BldgType          0
HouseStyle        0
OverallQual       0
OverallCond       0
YearBuilt         0
YearRemod/Add     0
RoofStyle         0
RoofMatl          0
Exterior1st       0
Exterior2nd       0
MasVnrType        0
MasVnrArea        0
ExterQual         0
ExterCond         0
Foundation        0
BsmtQual         25
BsmtCond         25
BsmtExposure     25
BsmtFinType1     25
BsmtFinSF1        0
BsmtFinType2     25
BsmtFinSF2        0
BsmtUnfSF         0
TotalBsmtSF       0
Heating           0
HeatingQC         0
CentralAir        0
Electrical        1
1stFlrSF          0
2ndFlrSF          0
LowQualFinSF      0
GrLivArea         0
BsmtFullBath      0
BsmtHalfBath      0
FullBath          0


### Null Values in Basement
___
Since **BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinSF1, BsmtFinType2, BsmtUnfSF, and TotalBsmtSF** columns are all related to basement, it would be easier to find out what unique values each have and group it together to do the null value replacement as a function.

In [1838]:
#defining the unique values in column
new_test_df['BsmtQual'].value_counts(dropna=False)

TA     396
Gd     355
Ex      73
Fa      28
NaN     25
Po       1
Name: BsmtQual, dtype: int64

In [1839]:
#defining the unique values in column
new_test_df['BsmtCond'].value_counts(dropna=False)

TA     781
Fa      39
Gd      33
NaN     25
Name: BsmtCond, dtype: int64

In [1840]:
#defining the unique values in column
new_test_df['BsmtExposure'].value_counts(dropna=False)

No     567
Av     130
Gd      80
Mn      76
NaN     25
Name: BsmtExposure, dtype: int64

In [1841]:
#defining the unique values in column
new_test_df['BsmtFinType1'].value_counts(dropna=False)

Unf    248
GLQ    243
ALQ    136
Rec    105
BLQ     69
LwQ     52
NaN     25
Name: BsmtFinType1, dtype: int64

In [1842]:
#defining the unique values in column
new_test_df['BsmtFinSF1'].value_counts(dropna=False, ascending=False)

0       273
24       10
662       4
16        4
936       4
600       4
602       4
504       3
767       3
329       3
480       3
483       3
595       3
500       3
520       3
250       3
560       3
168       3
192       3
196       3
288       3
375       3
220       3
734       2
739       2
724       2
324       2
334       2
1360      2
306       2
712       2
352       2
353       2
310       2
1300      2
300       2
284       2
280       2
697       2
267       2
777       2
256       2
779       2
793       2
1258      2
799       2
1249      2
224       2
698       2
399       2
370       2
552       2
474       2
584       2
484       2
489       2
663       2
564       2
550       2
468       2
528       2
1573      2
539       2
540       2
544       2
546       2
588       2
593       2
672       2
646       2
378       2
660       2
656       2
216       2
400       2
402       2
643       2
445       2
641       2
414       2
633       2
632       2
426       2
616 

In [1843]:
#defining the unique values in column
new_test_df['BsmtFinType2'].value_counts(dropna=False)

Unf    749
LwQ     29
Rec     26
NaN     25
BLQ     20
ALQ     18
GLQ     11
Name: BsmtFinType2, dtype: int64

In [1844]:
#defining the unique values in column
new_test_df['BsmtUnfSF'].value_counts(dropna=False, ascending=False)

0       79
270      7
840      5
572      5
100      5
384      5
396      5
416      4
108      4
115      4
186      4
728      4
392      4
641      4
600      4
490      4
491      4
440      4
546      4
150      3
244      3
780      3
784      3
168      3
422      3
132      3
130      3
894      3
536      3
1530     3
912      3
264      3
720      3
210      3
294      3
300      3
322      3
342      3
346      3
356      3
585      3
371      3
441      3
625      3
598      3
404      3
616      3
408      3
925      3
525      3
941      3
570      2
448      2
768      2
1339     2
230      2
229      2
316      2
30       2
226      2
624      2
224      2
590      2
218      2
785      2
321      2
982      2
960      2
204      2
1226     2
390      2
200      2
197      2
196      2
456      2
193      2
672      2
216      2
816      2
594      2
756      2
460      2
596      2
405      2
427      2
284      2
410      2
72       2
411      2
278      2
612      2

In [1845]:
#defining the unique values in column
new_test_df['TotalBsmtSF'].value_counts(dropna=False, ascending=False)

0       25
864     23
768     10
780      9
912      9
672      9
960      7
840      7
848      7
728      7
894      7
720      7
832      7
546      7
816      6
936      6
630      6
384      6
1056     5
952      5
756      5
1040     5
1008     5
1145     4
858      4
1248     4
1728     4
876      4
784      4
796      4
992      4
691      4
483      4
975      4
938      4
941      4
804      3
1232     3
1114     3
1226     3
723      3
1390     3
600      3
1216     3
1212     3
1176     3
1530     3
1026     3
1108     3
765      3
884      3
925      3
882      3
980      3
1024     3
856      3
920      3
972      3
836      3
1104     3
1064     3
585      2
1604     2
596      2
958      2
1434     2
1573     2
547      2
973      2
2002     2
970      2
1492     2
982      2
1482     2
827      2
984      2
1418     2
1284     2
1288     2
1300     2
1302     2
1313     2
1022     2
1314     2
1838     2
1020     2
1360     2
346      2
1392     2
988      2
1417     2

In [1846]:
#defining the unique values in column
new_test_df['BsmtFullBath'].value_counts(dropna=False)

0    507
1    356
2     15
Name: BsmtFullBath, dtype: int64

In [1847]:
#defining the unique values in column
new_test_df['BsmtHalfBath'].value_counts(dropna=False)

0    829
1     49
Name: BsmtHalfBath, dtype: int64

In [1848]:
#create column list for numerical and string 
bsmt_num = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF'\
           , 'BsmtFullBath', 'BsmtHalfBath']
bsmt_str = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1'\
            ,'BsmtFinType2']

In [1849]:
#for loop to replace null value to zero value within number value
for number in bsmt_num:
    new_test_df[number] = new_test_df[number].replace(np.nan, 0)

___
**'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', and 'BsmtFinType2'**  can have the same principle like what was done with LotFrontage.  A deeper studying needs to be done
___

In [1850]:
new_test_df[new_test_df.loc[:, new_test_df.isna().any()].select_dtypes(include='object').columns].isna().sum()

BsmtQual        25
BsmtCond        25
BsmtExposure    25
BsmtFinType1    25
BsmtFinType2    25
Electrical       1
GarageType      44
GarageFinish    45
GarageQual      45
GarageCond      45
dtype: int64

In [1851]:
#find the null difference
new_test_df[(new_test_df['BsmtExposure'].isna()) & (new_test_df['BsmtQual'].notna())]

,Id,PID,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemod/Add,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType


In [1852]:
#drop the three above rows
new_test_df.drop(new_test_df[(new_test_df['BsmtExposure'].isna()) & (new_test_df['BsmtQual'].notna())].index, inplace=True)

In [1853]:
#find the null difference for BsmtFinType2
new_test_df[(new_test_df['BsmtFinType2'].isna()) & (new_test_df['BsmtQual'].notna())]

,Id,PID,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemod/Add,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType


In [1854]:
#drop the above rows
new_test_df.drop(new_test_df[(new_test_df['BsmtFinType2'].isna()) & (new_test_df['BsmtQual'].notna())].index, inplace=True)

In [1855]:
#check the update
new_test_df[new_test_df.loc[:, new_test_df.isna().any()].select_dtypes(include='object').columns].isna().sum()

BsmtQual        25
BsmtCond        25
BsmtExposure    25
BsmtFinType1    25
BsmtFinType2    25
Electrical       1
GarageType      44
GarageFinish    45
GarageQual      45
GarageCond      45
dtype: int64

In [1856]:
#for loop to replace null value to NA value within string value
for string in bsmt_str:
    new_test_df[string] = new_test_df[string].replace(np.nan, 'NoBasement')

In [1857]:
#find all the notable nulls
test_df.isnull().sum()

Id                0
PID               0
MSSubClass        0
MSZoning          0
LotFrontage       0
LotArea           0
Street            0
LotShape          0
LandContour       0
Utilities         0
LotConfig         0
LandSlope         0
Neighborhood      0
Condition1        0
Condition2        0
BldgType          0
HouseStyle        0
OverallQual       0
OverallCond       0
YearBuilt         0
YearRemod/Add     0
RoofStyle         0
RoofMatl          0
Exterior1st       0
Exterior2nd       0
MasVnrType        0
MasVnrArea        0
ExterQual         0
ExterCond         0
Foundation        0
BsmtQual          0
BsmtCond          0
BsmtExposure      0
BsmtFinType1      0
BsmtFinSF1        0
BsmtFinType2      0
BsmtFinSF2        0
BsmtUnfSF         0
TotalBsmtSF       0
Heating           0
HeatingQC         0
CentralAir        0
Electrical        1
1stFlrSF          0
2ndFlrSF          0
LowQualFinSF      0
GrLivArea         0
BsmtFullBath      0
BsmtHalfBath      0
FullBath          0


### Null Values in Garage

In [1858]:
#defining the unique values in column
new_test_df['GarageYrBlt'].value_counts(dropna=False)

NaN       45
2005.0    37
2006.0    35
2007.0    30
2004.0    27
2003.0    26
1977.0    20
1950.0    19
1997.0    18
2008.0    17
1993.0    16
1974.0    16
1968.0    16
1960.0    16
1999.0    15
1976.0    15
1969.0    14
1957.0    14
1980.0    14
2000.0    14
2001.0    14
1994.0    14
1998.0    14
1959.0    13
2002.0    13
1963.0    12
1920.0    12
1970.0    12
2009.0    12
1954.0    12
1978.0    11
1961.0    11
1972.0    10
1967.0    10
1956.0    10
1962.0    10
1964.0    10
1958.0    10
1966.0    10
1979.0    10
1996.0     9
1973.0     9
1955.0     9
1995.0     9
1925.0     9
1984.0     8
1926.0     7
1991.0     7
1930.0     7
1989.0     7
1965.0     7
1971.0     6
1952.0     6
1985.0     6
1988.0     6
1939.0     5
1910.0     5
1951.0     5
1990.0     5
1992.0     5
1981.0     5
1948.0     5
1940.0     5
1938.0     5
1941.0     4
1983.0     4
1949.0     4
1946.0     4
1953.0     4
1924.0     4
1986.0     3
1915.0     3
1982.0     3
2010.0     3
1975.0     3
1900.0     3
1922.0     2

In [1859]:
new_test_df['GarageYrBlt'].describe()

count     833.000000
mean     1976.759904
std        25.689763
min      1900.000000
25%      1960.000000
50%      1978.000000
75%      2001.000000
max      2010.000000
Name: GarageYrBlt, dtype: float64

In [1860]:
#defining the unique values in column
new_test_df['GarageArea'].value_counts(dropna=False)

0       44
576     28
440     26
484     24
240     24
528     19
264     18
308     16
480     15
400     15
336     11
286     11
420     11
384      8
495      8
506      8
288      7
216      7
280      6
390      6
460      6
672      6
462      6
360      6
300      5
588      5
315      5
525      5
520      5
352      5
252      5
550      4
564      4
502      4
492      4
320      4
434      4
312      4
544      4
540      4
396      4
299      4
539      4
451      4
642      4
200      4
648      3
880      3
580      3
644      3
472      3
432      3
473      3
379      3
437      3
530      3
504      3
409      3
527      3
511      3
730      3
834      3
570      3
318      3
294      3
470      3
608      3
441      3
624      3
461      3
615      3
625      3
297      3
905      2
762      2
500      2
512      2
513      2
758      2
515      2
784      2
816      2
497      2
850      2
490      2
486      2
864      2
483      2
900      2
870      2
746      2

In [1861]:
#create a list
garage_num = ['GarageYrBlt', 'GarageCars', 'GarageArea' ]

#create a for loop and replace null values with '0'
#for loop to replace null value to zero value within numerial value
for number in garage_num:
    new_test_df[number] = new_test_df[number].replace(np.nan, 0)

___
Using the same principle for basement but for garage. Instead of **NoBasement**, it'll be called **NoGarage**
___

In [1862]:
#find the null difference
new_test_df[(new_test_df['GarageFinish'].isna()) & (new_test_df['GarageType'].notna())]

,Id,PID,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemod/Add,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType
764,1357,903426160,60,RM,57.0,8094,Pave,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2.5Unf,6,8,1910,1983,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,PConc,TA,TA,Mn,Rec,196,Unf,0,1046,1242,GasA,Gd,Y,SBrkr,1242,742,0,1984,0,0,2,0,5,1,TA,8,Typ,0,NoFirePlace,Detchd,0.0,NaN,1,360,NaN,NaN,Y,64,0,180,0,0,0,1000,9,2008,WD


In [1863]:
#find the null difference
new_test_df[(new_test_df['GarageQual'].isna()) & (new_test_df['GarageType'].notna())]

,Id,PID,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemod/Add,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType
764,1357,903426160,60,RM,57.0,8094,Pave,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2.5Unf,6,8,1910,1983,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,PConc,TA,TA,Mn,Rec,196,Unf,0,1046,1242,GasA,Gd,Y,SBrkr,1242,742,0,1984,0,0,2,0,5,1,TA,8,Typ,0,NoFirePlace,Detchd,0.0,NaN,1,360,NaN,NaN,Y,64,0,180,0,0,0,1000,9,2008,WD


In [1864]:
#find the null difference
new_test_df[(new_test_df['GarageCond'].isna()) & (new_test_df['GarageType'].notna())]

,Id,PID,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemod/Add,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType
764,1357,903426160,60,RM,57.0,8094,Pave,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2.5Unf,6,8,1910,1983,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,PConc,TA,TA,Mn,Rec,196,Unf,0,1046,1242,GasA,Gd,Y,SBrkr,1242,742,0,1984,0,0,2,0,5,1,TA,8,Typ,0,NoFirePlace,Detchd,0.0,NaN,1,360,NaN,NaN,Y,64,0,180,0,0,0,1000,9,2008,WD


In [1865]:
#drop the above rows
new_test_df.drop(new_test_df[(new_test_df['GarageFinish'].isna()) & (new_test_df['GarageType'].notna())].index, inplace=True)
new_test_df.drop(new_test_df[(new_test_df['GarageQual'].isna()) & (new_test_df['GarageType'].notna())].index, inplace=True)
new_test_df.drop(new_test_df[(new_test_df['GarageCond'].isna()) & (new_test_df['GarageType'].notna())].index, inplace=True)

In [1866]:
#check the update
new_test_df[new_test_df.loc[:, new_test_df.isna().any()].select_dtypes(include='object').columns].isna().sum()

Electrical       1
GarageType      44
GarageFinish    44
GarageQual      44
GarageCond      44
dtype: int64

In [1867]:
#create list
garage_cat_null = ['GarageType','GarageFinish','GarageQual','GarageCond']

#create for loop to replace null values to 'NoGarage'
for col in garage_cat_null: 
    new_test_df[col] = new_test_df[col].fillna('NoGarage')

In [1868]:
#check to make sure there are no more null values left
new_test_df[new_test_df.loc[:, new_test_df.isna().any()].select_dtypes(include='object').columns].isna().sum()

Electrical    1
dtype: int64

In [1869]:
#a complete check for any null values
new_test_df.isnull().sum().sum()

1

In [1870]:
#defining the unique values in column
new_test_df['Electrical'].value_counts(dropna=False)

SBrkr    812
FuseA     48
FuseF     15
NaN        1
FuseP      1
Name: Electrical, dtype: int64

In [1871]:
#convert the null value to None value because it has the biggest percentage 
new_test_df['Electrical'] = new_test_df['Electrical'].replace(np.nan, 'SBrkr')

In [1872]:
#defining the unique values in column
new_test_df['Electrical'].value_counts(dropna=False)

SBrkr    813
FuseA     48
FuseF     15
FuseP      1
Name: Electrical, dtype: int64

In [1889]:
#a complete check for any null values
new_test_df.isnull().sum().sum()

0

## Converting Ordinal Data to Numerical 
___

In [1874]:
#the list
#group columns that are string type
obj_cols = list(new_test_df.select_dtypes(include='object').columns)
print(obj_cols)
print('\n')
print(f"There are : {len(obj_cols)} string columns")

['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType']


There are : 38 string columns


#### Breaking the string valued into ordinal and nominal data types

In [1875]:
#make a list of relevant columns for ordinal
ord_col_cats = ['LotShape',
                'Utilities',
                'LandSlope',
                'ExterQual',
                'ExterCond',
                'BsmtQual',
                'BsmtCond',
                'BsmtExposure',
                'BsmtFinType1',
                'BsmtFinType2',
                'HeatingQC',
                'Electrical',
                'KitchenQual',
                'Functional',
                'FireplaceQu',
                'GarageFinish',
                'GarageQual',
                'GarageCond',
                'PavedDrive',]

In [1876]:
#change the order of the ordial encoder
lot_shape_cats = ['Reg', 'IR1', 'IR2', 'IR3']
utilities_cats = ['ELO', 'NoSeWa', 'NoSewr', 'AllPub']
land_slope_cats = ['Sev', 'Mod', 'Gtl']
exter_qual_cats = ['Po', 'Fa', 'TA', 'Gd', 'Ex']
exter_cond_cats = ['Po', 'Fa', 'TA', 'Gd', 'Ex']
bsmt_qual_cats = ['NoBasement', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
bsmt_cond_cats = ['NoBasement', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
bsmt_exposure_cats = ['NoBasement', 'No', 'Mn', 'Av', 'Gd']
bsmtfin_type1_cats = ['NoBasement', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']
bsmtfin_type2_cats = ['NoBasement', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']
heating_qc_cats = ['Po', 'Fa', 'TA', 'Gd', 'Ex']
electrical_cats = ['Mix', 'FuseP', 'FuseF', 'FuseA', 'SBrkr']
kitchen_qual_cats = ['Po', 'Fa', 'TA', 'Gd', 'Ex']
functional_cats = ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ']
fireplace_qu_cats = ['NoFirePlace', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
garage_finish_cats = ['NoGarage', 'Unf', 'RFn', 'Fin']
garage_qual_cats = ['NoGarage', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
garage_cond_cats = ['NoGarage', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
paved_drive_cats = ['N', 'P', 'Y']

In [1877]:
#map into a list
ord_category = [lot_shape_cats, 
                utilities_cats, 
                land_slope_cats, 
                exter_qual_cats, 
                exter_cond_cats, 
                bsmt_qual_cats, 
                bsmt_cond_cats, 
                bsmt_exposure_cats, 
                bsmtfin_type1_cats, 
                bsmtfin_type2_cats, 
                heating_qc_cats, 
                electrical_cats, 
                kitchen_qual_cats, 
                functional_cats,
                fireplace_qu_cats,
                garage_finish_cats, 
                garage_qual_cats, 
                garage_cond_cats, 
                paved_drive_cats]

In [1878]:
#use OrdinalEncoder on the columns
ord_to_num = OrdinalEncoder(categories = ord_category)
new_test_df[ord_col_cats] = ord_to_num.fit_transform(new_test_df[ord_col_cats])

In [1879]:
#check updated dataframe after OrdinalEncoder was applied
new_test_df.head()

,Id,PID,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemod/Add,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType
0,2658,902301120,190,RM,69.0,9142,Pave,0.0,Lvl,3.0,Inside,2.0,OldTown,Norm,Norm,2fmCon,2Story,6,8,1910,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,2.0,1.0,Stone,2.0,3.0,1.0,1.0,0,1.0,0,1020,1020,GasA,3.0,N,1.0,908,1020,0,1928,0,0,2,0,4,2,1.0,9,7.0,0,0.0,Detchd,1910.0,1.0,1,440,1.0,1.0,2.0,0,60,112,0,0,0,0,4,2006,WD
1,2718,905108090,90,RL,66.0,9662,Pave,1.0,Lvl,3.0,Inside,2.0,Sawyer,Norm,Norm,Duplex,1Story,5,4,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,2.0,2.0,CBlock,4.0,3.0,1.0,1.0,0,1.0,0,1967,1967,GasA,2.0,Y,4.0,1967,0,0,1967,0,0,2,0,6,2,2.0,10,7.0,0,0.0,Attchd,1977.0,3.0,2,580,3.0,3.0,2.0,170,0,0,0,0,0,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,1.0,Lvl,3.0,Inside,2.0,Gilbert,Norm,Norm,1Fam,2Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,3.0,2.0,PConc,4.0,4.0,3.0,6.0,554,1.0,0,100,654,GasA,4.0,Y,4.0,664,832,0,1496,1,0,2,1,3,1,3.0,7,7.0,1,4.0,Attchd,2006.0,2.0,2,426,3.0,3.0,2.0,100,24,0,0,0,0,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,0.0,Lvl,3.0,Inside,2.0,OldTown,Norm,Norm,1Fam,1Story,5,6,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,3.0,2.0,CBlock,3.0,3.0,1.0,1.0,0,1.0,0,968,968,GasA,2.0,Y,4.0,968,0,0,968,0,0,1,0,2,1,2.0,5,7.0,0,0.0,Detchd,1935.0,1.0,2,480,2.0,3.0,0.0,0,0,184,0,0,0,0,7,2007,WD
4,625,535105100,20,RL,66.0,9500,Pave,1.0,Lvl,3.0,Inside,2.0,NAmes,Norm,Norm,1Fam,1Story,6,5,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,2.0,2.0,CBlock,4.0,3.0,1.0,4.0,609,1.0,0,785,1394,GasA,3.0,Y,4.0,1394,0,0,1394,1,0,1,1,3,1,2.0,6,7.0,2,4.0,Attchd,1963.0,2.0,2,514,3.0,3.0,2.0,0,76,0,0,185,0,0,7,2009,WD


In [1880]:
#make a list of relevant columns for categorical

nominal_col_cats = ['MSZoning',
 'Street',
 'LandContour',
 'LotConfig',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'Foundation',
 'Heating',
 'CentralAir',
 'Functional',
 'SaleType', 'GarageType']

In [1881]:
new_test_df.head()

,Id,PID,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemod/Add,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType
0,2658,902301120,190,RM,69.0,9142,Pave,0.0,Lvl,3.0,Inside,2.0,OldTown,Norm,Norm,2fmCon,2Story,6,8,1910,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,2.0,1.0,Stone,2.0,3.0,1.0,1.0,0,1.0,0,1020,1020,GasA,3.0,N,1.0,908,1020,0,1928,0,0,2,0,4,2,1.0,9,7.0,0,0.0,Detchd,1910.0,1.0,1,440,1.0,1.0,2.0,0,60,112,0,0,0,0,4,2006,WD
1,2718,905108090,90,RL,66.0,9662,Pave,1.0,Lvl,3.0,Inside,2.0,Sawyer,Norm,Norm,Duplex,1Story,5,4,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,2.0,2.0,CBlock,4.0,3.0,1.0,1.0,0,1.0,0,1967,1967,GasA,2.0,Y,4.0,1967,0,0,1967,0,0,2,0,6,2,2.0,10,7.0,0,0.0,Attchd,1977.0,3.0,2,580,3.0,3.0,2.0,170,0,0,0,0,0,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,1.0,Lvl,3.0,Inside,2.0,Gilbert,Norm,Norm,1Fam,2Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,3.0,2.0,PConc,4.0,4.0,3.0,6.0,554,1.0,0,100,654,GasA,4.0,Y,4.0,664,832,0,1496,1,0,2,1,3,1,3.0,7,7.0,1,4.0,Attchd,2006.0,2.0,2,426,3.0,3.0,2.0,100,24,0,0,0,0,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,0.0,Lvl,3.0,Inside,2.0,OldTown,Norm,Norm,1Fam,1Story,5,6,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,3.0,2.0,CBlock,3.0,3.0,1.0,1.0,0,1.0,0,968,968,GasA,2.0,Y,4.0,968,0,0,968,0,0,1,0,2,1,2.0,5,7.0,0,0.0,Detchd,1935.0,1.0,2,480,2.0,3.0,0.0,0,0,184,0,0,0,0,7,2007,WD
4,625,535105100,20,RL,66.0,9500,Pave,1.0,Lvl,3.0,Inside,2.0,NAmes,Norm,Norm,1Fam,1Story,6,5,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,2.0,2.0,CBlock,4.0,3.0,1.0,4.0,609,1.0,0,785,1394,GasA,3.0,Y,4.0,1394,0,0,1394,1,0,1,1,3,1,2.0,6,7.0,2,4.0,Attchd,1963.0,2.0,2,514,3.0,3.0,2.0,0,76,0,0,185,0,0,7,2009,WD


In [1882]:
#get dummies for nominal
full_num_test_df = pd.get_dummies(new_test_df, columns=nominal_col_cats, drop_first=False)

In [1883]:
#check to see the dtypes
full_num_test_df.dtypes.value_counts()

uint8      149
int64       35
float64     21
dtype: int64

In [1885]:
#convert the floats to int64
for col in full_num_test_df.columns:
    if full_num_test_df[col].dtypes == 'float64':
        full_num_test_df[col] = full_num_test_df[col].astype('int64')

In [1886]:
#check again to see the dtypes
full_num_test_df.dtypes.value_counts()

uint8    149
int64     56
dtype: int64

In [1887]:
full_num_test_df.head()

,Id,PID,MSSubClass,LotFrontage,LotArea,LotShape,Utilities,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemod/Add,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_PreCast,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_PreCast,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_CBlock,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,CentralAir_N,CentralAir_Y,Functional_2.0,Functional_3.0,Functional_4.0,Functional_5.0,Functional_6.0,Functional_7.0,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_VWD,SaleType_WD,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NoGarage
0,2658,902301120,190,69,9142,0,3,2,6,8,1910,1950,0,2,1,2,3,1,1,0,1,0,1020,1020,3,1,908,1020,0,1928,0,0,2,0,4,2,1,9,0,0,1910,1,1,440,1,1,2,0,60,112,0,0,0,0,4,2006,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
1,2718,905108090,90,66,9662,1,3,2,5,4,1977,1977,0,2,2,4,3,1,1,0,1,0,1967,1967,2,4,1967,0,0,1967,0,0,2,0,6,2,2,10,0,0,1977,3,2,580,3,3,2,170,0,0,0,0,0,0,8,2006,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0

In [1891]:
#a complete check for any null values
full_num_test_df.isnull().sum().sum()

0

In [1894]:
len(full_num_test_df)

877

## Exporting Data
___
Once the file is completely cleaned. It can be exported for storage or future use.

In [1892]:
full_num_test_df.to_csv('../Data/full number test dataframe.csv', index=False)

[^ Return to Contents](#Contents:)